In [1]:
import datetime as dt
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import yfinance as yf
import OptionLib as opt
from datetime import datetime
import datetime

In [2]:
def opt_vol(market_price, spot, strike, r=0.05, time=1, volatility_candidates = np.arange(0.01,4,0.0001)):
    price_differences = np.zeros_like(volatility_candidates)
    for i in range(len(volatility_candidates)):

        candidate = volatility_candidates[i]

        price_differences[i] = market_price - opt.BSM(r, spot, strike, time, candidate, type='call')

    idx = np.argmin(abs(price_differences))
    implied_volatility = volatility_candidates[idx]
    return implied_volatility

In [3]:
"""ticker = 'TSLA'
hist = yf.Ticker(ticker).history(period="1mo")[['Close']]
hist = hist.rename(columns={'Close': 'Spot'})
hist"""

'ticker = \'TSLA\'\nhist = yf.Ticker(ticker).history(period="1mo")[[\'Close\']]\nhist = hist.rename(columns={\'Close\': \'Spot\'})\nhist'

In [3]:
df = opt.yf_get_chains('JPM')
df # JPM230721C00140000 # JPM230428P00124000

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,optionType,expDate,daysToExp
0,JPM230421C00065000,2023-04-04 19:47:43+00:00,65.0,63.10,76.05,76.45,0.000000,0.00000,4.0,0.0,5.855471,True,REGULAR,USD,call,2023-04-21 23:59:59,2
1,JPM230421C00070000,2023-04-11 19:07:19+00:00,70.0,58.70,70.95,71.35,0.000000,0.00000,61.0,31.0,5.179691,True,REGULAR,USD,call,2023-04-21 23:59:59,2
2,JPM230421C00075000,2023-04-04 19:48:33+00:00,75.0,53.15,66.05,66.30,0.000000,0.00000,5.0,0.0,4.765629,True,REGULAR,USD,call,2023-04-21 23:59:59,2
3,JPM230421C00080000,2023-04-20 14:02:52+00:00,80.0,61.09,61.00,61.20,2.439999,4.16027,2.0,5.0,4.218755,True,REGULAR,USD,call,2023-04-21 23:59:59,2
4,JPM230421C00085000,2023-04-04 18:23:49+00:00,85.0,43.10,56.05,56.35,0.000000,0.00000,83.0,0.0,3.964844,True,REGULAR,USD,call,2023-04-21 23:59:59,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,JPM251219P00160000,2023-03-23 15:12:58+00:00,160.0,37.14,27.10,28.45,0.000000,0.00000,NaN,2.0,0.175149,True,REGULAR,USD,put,2025-12-19 23:59:59,975
19,JPM251219P00170000,2023-03-17 19:49:51+00:00,170.0,48.42,34.20,36.65,0.000000,0.00000,2.0,0.0,0.182900,True,REGULAR,USD,put,2025-12-19 23:59:59,975
20,JPM251219P00190000,2023-04-13 18:28:42+00:00,190.0,61.08,47.40,50.95,0.000000,0.00000,NaN,0.0,0.139871,True,REGULAR,USD,put,2025-12-19 23:59:59,975
21,JPM251219P00195000,2023-03-15 18:26:42+00:00,195.0,66.05,54.10,58.40,0.000000,0.00000,NaN,0.0,0.193917,True,REGULAR,USD,put,2025-12-19 23:59:59,975


# Constructing the Portfolio

In [19]:
df_portfolio = pd.DataFrame({'contractSymbol': [],
                             'underlying': [],
                             'expDate': [],
                             'daysToExp': [],
                             'optionType': [],
                             'strike': [],
                             # contract size
                             'position': [],
                             'number_of_contracts': [],
                             'delta': [],
                             'delta_pos': [],
                             'DH_quantity': []
                             }
                            )

In [10]:
def create_portfolio():
    return pd.DataFrame({'contractSymbol': [],
                             'underlying': [],
                             'expDate': [],
                             'daysToExp': [],
                             'optionType': [],
                             'strike': [],
                             # contract size
                             'position': [],
                             'number_of_contracts': [],
                             'delta': [],
                             'delta_pos': [],
                             'DH_quantity': []
                             }
                            )

In [20]:
df_portfolio

,contractSymbol,underlying,expDate,daysToExp,optionType,strike,position,number_of_contracts,delta,delta_pos,DH_quantity


In [14]:
import datetime

In [4]:
def add_option(option_chain, df_portfolio, contract, number_of_contracts, spot, position=1,  r=0.05):
    """
    Performs operations of long/short for needed option, computes delta, and hedge quantity

    :param option_chain: the chain we got option from
    :param df_portfolio: portfolio we add option to
    :param contract: symbol of needed one for ex. 'JPM230421C00110000'
    :param number_of_contracts: number of contract in operation
    :param position: 1 for long, -1 for short
    :return: returns updated df_portfolio
    """
    option = option_chain[ option_chain['contractSymbol'] == contract ].reset_index(drop=True).to_dict()


    delta = opt.delta(r=r,
                      spot=spot,
                      strike=option['strike'][0],
                      time=option['daysToExp'][0],
                      sigma=option['impliedVolatility'][0],
                      # can change to our formula # iv = opt_vol(market_price=market_price, spot=spot, strike=strike, r=r, time=time)
                      type=option['optionType'][0])

    delta_pos = delta * 100 * number_of_contracts
    DH_quantity = position * abs(delta) * 100 * number_of_contracts

    df_portfolio.loc[len(df_portfolio.index)] = [
                                             option['contractSymbol'][0],
                                             'JPM',
                                             option['expDate'][0], # datetime.strptime( _ , '%Y-%m-%d %H:%M:%S')
                                             option['daysToExp'][0],
                                             option['optionType'][0],
                                             option['strike'][0],
                                             position,
                                             number_of_contracts,
                                             delta,
                                             delta_pos,
                                             DH_quantity
                                            ]
    return df_portfolio

In [21]:
df_portfolio = add_option(option_chain=df, df_portfolio=df_portfolio, contract='JPM230428P00124000', number_of_contracts=8, spot=141.38, position=1 )
df_portfolio

,contractSymbol,underlying,expDate,daysToExp,optionType,strike,position,number_of_contracts,delta,delta_pos,DH_quantity
0,JPM230428P00124000,JPM,2023-04-28 23:59:59,10,put,124.0,1,8,-0.130474,-104.379573,104.379573


In [5]:
def rebalance(df_portfolio, market_price, spot, r=0.05):

    for contract in list(df_portfolio['contractSymbol']):

        option = df_portfolio[ df_portfolio['contractSymbol'] == contract ].reset_index(drop=True).to_dict()
        #option = df_portfolio[ df_portfolio['contractSymbol'] == 'JPM230421C00110000' ].reset_index(drop=True).to_dict()

        strike = option['strike'][0]
        time = (option['expDate'][0] - datetime.datetime.today()).days + 1
        type = option['optionType'][0]
        number_of_contracts = option['number_of_contracts'][0]
        position = option['position'][0]

        iv = opt_vol(market_price=market_price, spot=spot, strike=strike, r=r, time=time)

        #opt.imp_vol(market_price=market_price, spot=spot, strike=strike, time=time, r=r, type=type, precision=1.0e-5, iterations=200)
        # here can get data from moex/yf directly
        #print(iv)

        delta = opt.delta(r=r,
                      spot=spot,
                      strike=strike,
                      time=time,
                      sigma=iv,  # can change to our formula
                      type=type)


        delta_pos = delta * 100 * number_of_contracts
        DH_quantity = position * abs(delta) * 100 * number_of_contracts

        df_portfolio.loc[df_portfolio['contractSymbol'] == contract] = [option['contractSymbol'][0],
                                             'JPM',
                                             option['expDate'][0], # datetime.strptime( _ , '%Y-%m-%d %H:%M:%S')
                                             time,
                                             type,
                                             strike,
                                             position,
                                             number_of_contracts,
                                             delta,
                                             delta_pos,
                                             DH_quantity]
    return df_portfolio

In [22]:
df_portfolio = rebalance(df_portfolio=df_portfolio, market_price=6.8, spot=139, r=0.05)

In [23]:
df_portfolio

,contractSymbol,underlying,expDate,daysToExp,optionType,strike,position,number_of_contracts,delta,delta_pos,DH_quantity
0,JPM230428P00124000,JPM,2023-04-28 23:59:59,10,put,124.0,1,8,-6.858572e-17,-5.486858e-14,5.486858e-14


# Delta Hedging Simulaton

In [6]:
def gbm(n_years = 10, n_scenarios=1000, mu=0.07, sigma=0.15, steps_per_year=12, s_0=100.0, prices=True):
    """
    Evolution of Geometric Brownian Motion trajectories, such as for Stock Prices through Monte Carlo
    :param n_years:  The number of years to generate data for
    :param n_scenarios: The number of scenarios/trajectories aka number of columns in ret dataset
    :param mu: Annualized Drift, e.g. Market Return
    :param sigma: Annualized Volatility
    :param steps_per_year: granularity of the simulation
    :param s_0: initial value
    :return: a numpy array of n_paths columns and n_years*steps_per_year rows
    """
    # Derive per-step Model Parameters from User Specifications
    dt = 1/steps_per_year
    n_steps = int(n_years*steps_per_year) + 1
    # the standard way ...
    # rets_plus_1 = np.random.normal(loc=mu*dt+1, scale=sigma*np.sqrt(dt), size=(n_steps, n_scenarios))
    # without discretization error ...
    rets_plus_1 = np.random.normal(loc=(1+mu)**dt, scale=(sigma*np.sqrt(dt)), size=(n_steps, n_scenarios))
    rets_plus_1[0] = 1
    ret_val = s_0*pd.DataFrame(rets_plus_1).cumprod() if prices else rets_plus_1-1
    return ret_val

In [30]:
df_portfolio = create_portfolio()
df_portfolio = add_option(option_chain=df, df_portfolio=df_portfolio, contract='JPM230428C00145000', number_of_contracts=8, spot=141.38, position=1 )
df_portfolio

,contractSymbol,underlying,expDate,daysToExp,optionType,strike,position,number_of_contracts,delta,delta_pos,DH_quantity
0,JPM230428C00145000,JPM,2023-04-28 23:59:59,10,call,145.0,1,8,0.889051,711.241187,711.241187


In [42]:
scenario_underlying = gbm(n_scenarios=1, s_0=120, mu=.002)
option_scenario = gbm(n_scenarios=1, s_0=25, mu=-.002)

In [43]:
history = pd.DataFrame({'underlying_price': [], 'option_price': [], 'delta': [], 'delta_pos': [], 'DH_quantity': []})

for step in range(scenario_underlying.shape[0]):

    print(step)

    spot = float(scenario_underlying.iloc[step])
    option_market_price = float(option_scenario.iloc[step])
    rebalanced_df = rebalance(df_portfolio=df_portfolio, market_price=option_market_price, spot=spot)

    for contract in list(df_portfolio['contractSymbol']):
        option = df_portfolio[ df_portfolio['contractSymbol'] == contract].reset_index(drop=True).to_dict()
        history.loc[len(history.index)] = [ spot, option_market_price,  option['delta'][0], option['delta_pos'][0], option['DH_quantity'][0]]

    df_portfolio = rebalanced_df


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120


In [44]:
 history

,underlying_price,option_price,delta,delta_pos,DH_quantity
0,120.000000,25.000000,1.000000,800.000000,800.000000
1,124.916913,25.605208,1.000000,800.000000,800.000000
2,124.329545,23.428332,1.000000,800.000000,800.000000
3,117.594141,23.865859,1.000000,800.000000,800.000000
4,118.967166,23.980574,1.000000,800.000000,800.000000
...,...,...,...,...,...
116,43.283285,28.466316,0.801661,641.329079,641.329079
117,38.936785,26.338949,0.811040,648.831927,648.831927
118,41.809500,24.974322,0.761140,608.911725,608.911725
119,41.116829,25.818674,0.780930,624.743681,624.743681


In [ ]:
"""if oi - of / si - sf = delta why we calc it with bsm
if d h replicates bond why not bond use, for selling premium or just conv"""

In [11]:
df_portfolio = create_portfolio()
df_portfolio = add_option(option_chain=df, df_portfolio=df_portfolio, contract='JPM230428C00145000', number_of_contracts=8, spot=141.38, position=1 )
df_portfolio

,contractSymbol,underlying,expDate,daysToExp,optionType,strike,position,number_of_contracts,delta,delta_pos,DH_quantity
0,JPM230428C00145000,JPM,2023-04-28 23:59:59,9,call,145.0,1,8,0.865104,692.083287,692.083287


In [12]:
scenario_underlying = gbm(n_scenarios=1, s_0=120, mu=.002)
option_scenario = gbm(n_scenarios=1, s_0=25, mu=-.002)

In [17]:
history = pd.DataFrame({'underlying_price': [], 'option_price': [],
                        'delta': [], 'delta_pos': [],
                        'DH_quantity': [], 'underlying_needed': [], 'underlying_pos': []})

for step in range(scenario_underlying.shape[0]):

    print(step)

    spot = float(scenario_underlying.iloc[step])
    option_market_price = float(option_scenario.iloc[step])
    rebalanced_df = rebalance(df_portfolio=df_portfolio, market_price=option_market_price, spot=spot)

    for contract in list(df_portfolio['contractSymbol']):
        option = df_portfolio[ df_portfolio['contractSymbol'] == contract].reset_index(drop=True).to_dict()
        history.loc[len(history.index)] = [ spot, option_market_price,
                                            option['delta'][0], option['delta_pos'][0],
                                            option['DH_quantity'][0], option['DH_quantity'][0]/spot, -np.sign(option['delta_pos'][0])]

    df_portfolio = rebalanced_df


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120


In [18]:
history

,underlying_price,option_price,delta,delta_pos,DH_quantity,underlying_needed,underlying_pos
0,120.000000,25.000000,1.000000,800.000000,800.000000,6.666667,-1.0
1,110.080640,24.780115,0.747941,598.352480,598.352480,5.435583,-1.0
2,116.122338,27.083789,0.829282,663.425445,663.425445,5.713160,-1.0
3,113.456048,27.919812,0.769023,615.218568,615.218568,5.422528,-1.0
4,110.983356,28.541619,0.736963,589.570399,589.570399,5.312242,-1.0
...,...,...,...,...,...,...,...
116,61.418184,24.251426,0.642706,514.164786,514.164786,8.371540,-1.0
117,61.399870,23.375081,0.632300,505.840114,505.840114,8.238456,-1.0
118,62.095566,23.821407,0.635551,508.441182,508.441182,8.188043,-1.0
119,61.225318,24.921022,0.651187,520.949711,520.949711,8.508730,-1.0
